<a href="https://colab.research.google.com/github/LathaAlagar/latha2809/blob/main/23BIT050_Data_Preprocessing_Challenge_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark kagglehub

In [3]:
import kagglehub
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, when, count, isnan
from pyspark.ml.feature import VectorAssembler, StandardScaler

# Download latest version of dataset
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
print("📁 Path to dataset files:", path)


Using Colab cache for faster access to the 'creditcardfraud' dataset.
📁 Path to dataset files: /kaggle/input/creditcardfraud


In [4]:
spark = SparkSession.builder \
    .appName("DataPreprocessingChallenge") \
    .getOrCreate()

print("✅ Spark session created successfully!")


✅ Spark session created successfully!


In [5]:
# Read CSV file
file_path = f"{path}/creditcard.csv"  # Kaggle dataset file name
data = spark.read.csv(file_path, header=True, inferSchema=True)

print("✅ Dataset loaded successfully.")
data.printSchema()
data.show(5)


✅ Dataset loaded successfully.
root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable 

In [6]:
# Count missing values
missing_counts = data.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in data.columns])
print("🔍 Missing value count per column:")
missing_counts.show()

# Fill missing numeric columns with mean
numeric_cols = [c for c, t in data.dtypes if t in ['double', 'int']]
for column in numeric_cols:
    mean_value = data.select(mean(col(column))).collect()[0][0]
    data = data.fillna({column: mean_value})

# Fill missing categorical columns (if any)
categorical_cols = [c for c, t in data.dtypes if t == 'string']
for column in categorical_cols:
    data = data.fillna({column: 'Unknown'})

print("✅ Missing values handled.")


🔍 Missing value count per column:
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+
|Time| V1| V2| V3| V4| V5| V6| V7| V8| V9|V10|V11|V12|V13|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|V26|V27|V28|Amount|Class|
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+
|   0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|     0|    0|
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+

✅ Missing values handled.


In [7]:
for column in numeric_cols:
    data = data.withColumn(column, col(column).cast("double"))

data.printSchema()
print("✅ Data types standardized.")


root
 |-- Time: double (nullable = false)
 |-- V1: double (nullable = false)
 |-- V2: double (nullable = false)
 |-- V3: double (nullable = false)
 |-- V4: double (nullable = false)
 |-- V5: double (nullable = false)
 |-- V6: double (nullable = false)
 |-- V7: double (nullable = false)
 |-- V8: double (nullable = false)
 |-- V9: double (nullable = false)
 |-- V10: double (nullable = false)
 |-- V11: double (nullable = false)
 |-- V12: double (nullable = false)
 |-- V13: double (nullable = false)
 |-- V14: double (nullable = false)
 |-- V15: double (nullable = false)
 |-- V16: double (nullable = false)
 |-- V17: double (nullable = false)
 |-- V18: double (nullable = false)
 |-- V19: double (nullable = false)
 |-- V20: double (nullable = false)
 |-- V21: double (nullable = false)
 |-- V22: double (nullable = false)
 |-- V23: double (nullable = false)
 |-- V24: double (nullable = false)
 |-- V25: double (nullable = false)
 |-- V26: double (nullable = false)
 |-- V27: double (nullable = fa

In [8]:
before = data.count()
data = data.dropDuplicates()
after = data.count()

print(f"🧹 Removed {before - after} duplicate rows.")


🧹 Removed 1081 duplicate rows.


In [9]:
# Assemble all numeric columns into a single feature vector
assembler = VectorAssembler(
    inputCols=[c for c in data.columns if c not in ['Time', 'Class']],
    outputCol="features"
)
assembled = assembler.transform(data)

# Apply StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
scaler_model = scaler.fit(assembled)
scaled_data = scaler_model.transform(assembled)

print("✅ Data normalization completed.")
scaled_data.select("scaled_features").show(5, truncate=False)


✅ Data normalization completed.
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|scaled_features                                                                                                                                                                                                                                                                                                                                                                              

In [10]:
from pyspark.sql.functions import floor

# Create hour of transaction
scaled_data = scaled_data.withColumn("Transaction_Hour", floor(col("Time") / 3600))

# Create transaction category
scaled_data = scaled_data.withColumn(
    "Amount_Category",
    when(col("Amount") < 10, "Low") \
    .when(col("Amount") < 100, "Medium") \
    .otherwise("High")
)

scaled_data.select("Time", "Transaction_Hour", "Amount", "Amount_Category").show(5)
print("✅ Feature engineering completed.")


+------+----------------+------+---------------+
|  Time|Transaction_Hour|Amount|Amount_Category|
+------+----------------+------+---------------+
|  23.0|               0|  33.0|         Medium|
| 231.0|               0| 26.98|         Medium|
| 462.0|               0| 13.99|         Medium|
| 751.0|               0| 14.05|         Medium|
|1253.0|               0|  6.28|            Low|
+------+----------------+------+---------------+
only showing top 5 rows

✅ Feature engineering completed.


In [12]:
# Drop complex vector columns before saving
clean_data = scaled_data.drop("features", "scaled_features")

# Define output path
output_path = "/content/cleaned_creditcard_data.csv"

# Save as CSV
clean_data.write.csv(output_path, header=True, mode="overwrite")

print(f"✅ Cleaned dataset saved successfully at: {output_path}")


✅ Cleaned dataset saved successfully at: /content/cleaned_creditcard_data.csv


In [15]:
print("📊 Data Preprocessing Summary")
print("- Missing values handled (numeric: mean, categorical: 'Unknown')")
print("- Data types standardized to double")
print("- Duplicates removed")
print("- Features normalized using StandardScaler")
print("- Engineered features: Transaction_Hour, Amount_Category")
print("✅ Dataset ready for downstream analytics or ML tasks.")


📊 Data Preprocessing Summary
- Missing values handled (numeric: mean, categorical: 'Unknown')
- Data types standardized to double
- Duplicates removed
- Features normalized using StandardScaler
- Engineered features: Transaction_Hour, Amount_Category
✅ Dataset ready for downstream analytics or ML tasks.
